In [1]:
import pandas as pd
from sklearn import model_selection, ensemble, metrics

In [2]:
df_train = pd.read_csv('../../Feature_Encoding/data/train_count_encoding.csv')
df_test = pd.read_csv('../../Feature_Encoding/data/test_count_encoding.csv')
train = pd.read_csv('../../Feature_Engineering/data/other-cleaned_train.csv')

In [3]:
for column in df_test.columns.tolist():
    df_test[column] = df_test[column].fillna(0)  

df_test= df_test.replace('', 0)

In [4]:
def cross_val(model, x_train, y_train):
    score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
    print(score_cross_val.mean())

In [5]:
def test_model(model, x_test, y_test):
    predictions = model.predict_proba(x_test)[:,1]
    logloss = metrics.log_loss(y_test, predictions)
    accuracy = metrics.accuracy_score(y_test, predictions.round())
    print("Accuracy: %.2f%%, Logloss: %.2f" % (accuracy*100.0, logloss))

In [6]:
def RandomForest_con_gridsearch(x_train, y_train, x_validation, y_validation):

  rf = ensemble.RandomForestClassifier()
  params_rf = {'n_estimators': [50, 100, 150],'max_depth':[3,10,6,15]}
  rf_gs = model_selection.GridSearchCV(rf, params_rf, cv=5)
  rf_gs.fit(x_train, y_train)
  rf_best = rf_gs.best_estimator_

  print(rf_gs.best_params_)
  print('rf: {}'.format(rf_best.score(x_validation, y_validation)))

  return rf_best

In [7]:
y = train.Target
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(df_train, y, test_size=0.3, stratify=y)

In [8]:
rf = RandomForest_con_gridsearch(x_train, y_train, x_validation, y_validation)

{'max_depth': 15, 'n_estimators': 100}
rf: 0.9145113524185587


In [9]:
test_model(rf, x_validation, y_validation)

Accuracy: 91.45%, Logloss: 0.23


In [10]:
cross_val(rf, x_train, y_train)

0.9086986845280419


In [11]:
y_pred = rf.predict_proba(df_test)[:,1]
submission_rf = pd.DataFrame(data={'Opportunity_ID':df_test['Opportunity_ID'], 'Target': y_pred})
submission_rf = submission_rf.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_rf.to_csv('../submits/count_rf.csv', index=False)